In [1]:
import glob
import json
import pandas as pd

In [2]:
files = sorted(glob.glob('../data/coco_degradation/properties_*.json'))

In [58]:
def fix_df(df):
    # patches
    df['width'] = df['width'].astype(pd.Int16Dtype())
    df['height'] = df['height'].astype(pd.Int16Dtype())
    df['channels'] = df['channels'].astype(pd.Int16Dtype())
    return df


def sub_df(filename):
    data = json.load(open(filename))
    df = pd.DataFrame(data['files'])
    df['quality'] = data['quality']
    df = fix_df(df)
    return df

In [4]:
sub_df(files[81]).to_csv('test.csv', float_format='%.3f')
list(open('test.csv'))[:5]

[',image,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity,orig_shape,loss_shape,quality\n',
 '0,000000000139.jpg,640,426,3,21.241,34.859,0.919,,,82\n',
 '1,000000000285.jpg,586,640,3,40.106,32.099,0.942,,,82\n',
 '2,000000000632.jpg,640,483,3,32.658,32.991,0.956,,,82\n',
 '3,000000000724.jpg,375,500,3,22.061,34.694,0.948,,,82\n']

In [5]:
df = pd.concat([sub_df(f) for f in files])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   image                    500000 non-null  object 
 1   width                    487600 non-null  Int16  
 2   height                   487600 non-null  Int16  
 3   channels                 487600 non-null  Int16  
 4   mean_squared_error       487600 non-null  float64
 5   peak_signal_noise_ratio  487600 non-null  float64
 6   structural_similarity    487600 non-null  float64
 7   orig_shape               12400 non-null   object 
 8   loss_shape               12400 non-null   object 
 9   quality                  500000 non-null  int64  
dtypes: Int16(3), float64(3), int64(1), object(3)
memory usage: 34.8+ MB


In [10]:
bad = df[df.orig_shape.notnull()]
bad_img = bad.image.unique()

In [8]:
(bad.orig_shape.apply(lambda x: x[:2]) == bad.loss_shape).value_counts()

True    12400
dtype: int64

Reusing from *2020-06-28-JPEG-quality.ipynb*

In [53]:
import os
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error, structural_similarity
import numpy as np
import cv2

!if [ ! -d orig ] ; then unzip -d orig ~/val2017.zip ; fi

def set_up(directory='loss'):
    import shutil
    !rm -rf {directory}
    !mkdir -p {directory}/val2017
    for name in bad_img:
        shutil.copy2(f'orig/val2017/{name}', f'loss/val2017/{name}')

def pdegrade(q):
    set_up()
    !ls loss/val2017/* | xargs -n {len(bad_img) // 8} -P 8 mogrify -quality {q}
    print(f'Done degrading to {q}')
    
def evaluate(orig_path, loss_path):
    orig = cv2.imread(orig_path, cv2.IMREAD_ANYCOLOR)
    loss = cv2.imread(loss_path, cv2.IMREAD_ANYCOLOR)
    
    if orig.shape != loss.shape:
        print(orig.shape, loss.shape)
        loss = np.stack([loss, loss, loss], axis=-1)
        print('after:', orig.shape, loss.shape)
    
    return {
        'image': os.path.basename(orig_path),
        'width': orig.shape[1],
        'height': orig.shape[0],
        'channels': 1 if len(orig.shape) < 3 else orig.shape[2],
        'mean_squared_error': mean_squared_error(orig, loss),
        'peak_signal_noise_ratio': peak_signal_noise_ratio(orig, loss),
        'structural_similarity': 
            structural_similarity(orig, loss, multichannel=True) if len(orig.shape) > 2 
            else structural_similarity(orig, loss),
    }

results = []

In [54]:
%%time

for q in range(1, 101):
    pdegrade(q)
    for loss in jpgs('loss'):
        orig = loss.replace('loss', 'orig')
        part = evaluate(orig, loss)
        part['quality'] = q
        print(part)
        results.append(part)        

Done degrading to 1
(512, 640, 3) (512, 640)
after: (512, 640, 3) (512, 640, 3)
{'image': '000000001425.jpg', 'width': 640, 'height': 512, 'channels': 3, 'mean_squared_error': 211.87717895507814, 'peak_signal_noise_ratio': 24.869961789924897, 'structural_similarity': 0.6921483564459172, 'quality': 1}
(315, 640, 3) (315, 640)
after: (315, 640, 3) (315, 640, 3)
{'image': '000000001490.jpg', 'width': 640, 'height': 315, 'channels': 3, 'mean_squared_error': 468.140625, 'peak_signal_noise_ratio': 21.42704030255805, 'structural_similarity': 0.4787203925119889, 'quality': 1}
(302, 500, 3) (302, 500)
after: (302, 500, 3) (302, 500, 3)
{'image': '000000002299.jpg', 'width': 500, 'height': 302, 'channels': 3, 'mean_squared_error': 999.805761589404, 'peak_signal_noise_ratio': 18.131647257315194, 'structural_similarity': 0.5649575480549682, 'quality': 1}
(640, 443, 3) (640, 443)
after: (640, 443, 3) (640, 443, 3)
{'image': '000000003156.jpg', 'width': 443, 'height': 640, 'channels': 3, 'mean_squar

{'image': '000000112298.jpg', 'width': 640, 'height': 480, 'channels': 3, 'mean_squared_error': 204.471943359375, 'peak_signal_noise_ratio': 25.02446636202488, 'structural_similarity': 0.7282743316129544, 'quality': 1}
(425, 640, 3) (425, 640)
after: (425, 640, 3) (425, 640, 3)
{'image': '000000132587.jpg', 'width': 640, 'height': 425, 'channels': 3, 'mean_squared_error': 320.6999669117647, 'peak_signal_noise_ratio': 23.069814457478227, 'structural_similarity': 0.40473732115175737, 'quality': 1}
(436, 640, 3) (436, 640)
after: (436, 640, 3) (436, 640, 3)
{'image': '000000145620.jpg', 'width': 640, 'height': 436, 'channels': 3, 'mean_squared_error': 801.8099842316514, 'peak_signal_noise_ratio': 19.09008901034082, 'structural_similarity': 0.328704400472555, 'quality': 1}
(640, 448, 3) (640, 448)
after: (640, 448, 3) (640, 448, 3)
{'image': '000000148783.jpg', 'width': 448, 'height': 640, 'channels': 3, 'mean_squared_error': 1434.7529087611608, 'peak_signal_noise_ratio': 16.56303246976608

{'image': '000000255483.jpg', 'width': 452, 'height': 500, 'channels': 3, 'mean_squared_error': 449.69081858407077, 'peak_signal_noise_ratio': 21.601663402756856, 'structural_similarity': 0.5348815454915553, 'quality': 1}
(424, 640, 3) (424, 640)
after: (424, 640, 3) (424, 640, 3)
{'image': '000000255749.jpg', 'width': 640, 'height': 424, 'channels': 3, 'mean_squared_error': 206.34080557193397, 'peak_signal_noise_ratio': 24.984952391366026, 'structural_similarity': 0.6105310523339293, 'quality': 1}
(640, 523, 3) (640, 523)
after: (640, 523, 3) (640, 523, 3)
{'image': '000000261796.jpg', 'width': 523, 'height': 640, 'channels': 3, 'mean_squared_error': 823.1367919455067, 'peak_signal_noise_ratio': 18.976083469794457, 'structural_similarity': 0.5952945352056289, 'quality': 1}
(333, 500, 3) (333, 500)
after: (333, 500, 3) (333, 500, 3)
{'image': '000000278749.jpg', 'width': 500, 'height': 333, 'channels': 3, 'mean_squared_error': 732.1116096096096, 'peak_signal_noise_ratio': 19.4850306704

{'image': '000000407943.jpg', 'width': 640, 'height': 427, 'channels': 3, 'mean_squared_error': 328.1558950526932, 'peak_signal_noise_ratio': 22.97000150459155, 'structural_similarity': 0.5452817403439989, 'quality': 1}
(640, 544, 3) (640, 544)
after: (640, 544, 3) (640, 544, 3)
{'image': '000000415882.jpg', 'width': 544, 'height': 640, 'channels': 3, 'mean_squared_error': 327.7748104319853, 'peak_signal_noise_ratio': 22.97504785979795, 'structural_similarity': 0.6130277302922764, 'quality': 1}
(480, 640, 3) (480, 640)
after: (480, 640, 3) (480, 640, 3)
{'image': '000000417043.jpg', 'width': 640, 'height': 480, 'channels': 3, 'mean_squared_error': 803.6064615885417, 'peak_signal_noise_ratio': 19.080369407297702, 'structural_similarity': 0.4859903213656574, 'quality': 1}
(428, 640, 3) (428, 640)
after: (428, 640, 3) (428, 640, 3)
{'image': '000000427997.jpg', 'width': 640, 'height': 428, 'channels': 3, 'mean_squared_error': 276.7036945093458, 'peak_signal_noise_ratio': 23.71065403048568

{'image': '000000581482.jpg', 'width': 640, 'height': 427, 'channels': 3, 'mean_squared_error': 598.7161153395784, 'peak_signal_noise_ratio': 20.358594128794657, 'structural_similarity': 0.5958339384682955, 'quality': 1}
Done degrading to 2
(512, 640, 3) (512, 640)
after: (512, 640, 3) (512, 640, 3)
{'image': '000000001425.jpg', 'width': 640, 'height': 512, 'channels': 3, 'mean_squared_error': 211.8219207763672, 'peak_signal_noise_ratio': 24.871094590171367, 'structural_similarity': 0.692155279686906, 'quality': 2}
(315, 640, 3) (315, 640)
after: (315, 640, 3) (315, 640, 3)
{'image': '000000001490.jpg', 'width': 640, 'height': 315, 'channels': 3, 'mean_squared_error': 468.0991021825397, 'peak_signal_noise_ratio': 21.427425527196235, 'structural_similarity': 0.47872576582082066, 'quality': 2}
(302, 500, 3) (302, 500)
after: (302, 500, 3) (302, 500, 3)
{'image': '000000002299.jpg', 'width': 500, 'height': 302, 'channels': 3, 'mean_squared_error': 998.2828940397351, 'peak_signal_noise_rat

...

{'image': '000000474854.jpg', 'width': 424, 'height': 640, 'channels': 3, 'mean_squared_error': 0.07984964622641509, 'peak_signal_noise_ratio': 59.10807364529365, 'structural_similarity': 0.9995006545304385, 'quality': 99}
(491, 640, 3) (491, 640)
after: (491, 640, 3) (491, 640, 3)
{'image': '000000484978.jpg', 'width': 640, 'height': 491, 'channels': 3, 'mean_squared_error': 0.10559126782077394, 'peak_signal_noise_ratio': 57.89452356439603, 'structural_similarity': 0.9995510204295445, 'quality': 99}
(566, 640, 3) (566, 640)
after: (566, 640, 3) (566, 640, 3)
{'image': '000000489842.jpg', 'width': 640, 'height': 566, 'channels': 3, 'mean_squared_error': 0.10591596731448763, 'peak_signal_noise_ratio': 57.88118923953612, 'structural_similarity': 0.9997857021028346, 'quality': 99}
(320, 500, 3) (320, 500)
after: (320, 500, 3) (320, 500, 3)
{'image': '000000504389.jpg', 'width': 500, 'height': 320, 'channels': 3, 'mean_squared_error': 0.1065, 'peak_signal_noise_ratio': 57.85730753093154, '

{'image': '000000052565.jpg', 'width': 640, 'height': 458, 'channels': 3, 'mean_squared_error': 0.05345933406113537, 'peak_signal_noise_ratio': 60.85056816387077, 'structural_similarity': 0.9997969235660084, 'quality': 100}
(640, 359, 3) (640, 359)
after: (640, 359, 3) (640, 359, 3)
{'image': '000000052591.jpg', 'width': 359, 'height': 640, 'channels': 3, 'mean_squared_error': 0.05601061977715877, 'peak_signal_noise_ratio': 60.648099825517704, 'structural_similarity': 0.9995477668559128, 'quality': 100}
(486, 640, 3) (486, 640)
after: (486, 640, 3) (486, 640, 3)
{'image': '000000058393.jpg', 'width': 640, 'height': 486, 'channels': 3, 'mean_squared_error': 0.03847093621399177, 'peak_signal_noise_ratio': 62.27947605608064, 'structural_similarity': 0.999890338039071, 'quality': 100}
(640, 486, 3) (640, 486)
after: (640, 486, 3) (640, 486, 3)
{'image': '000000059635.jpg', 'width': 486, 'height': 640, 'channels': 3, 'mean_squared_error': 0.056584362139917695, 'peak_signal_noise_ratio': 60.

{'image': '000000221155.jpg', 'width': 640, 'height': 512, 'channels': 3, 'mean_squared_error': 0.057659912109375, 'peak_signal_noise_ratio': 60.52206384807441, 'structural_similarity': 0.9998666755860247, 'quality': 100}
(424, 640, 3) (424, 640)
after: (424, 640, 3) (424, 640, 3)
{'image': '000000222458.jpg', 'width': 640, 'height': 424, 'channels': 3, 'mean_squared_error': 0.05776090801886793, 'peak_signal_noise_ratio': 60.514463490126275, 'structural_similarity': 0.9998435644404627, 'quality': 100}
(426, 640, 3) (426, 640)
after: (426, 640, 3) (426, 640, 3)
{'image': '000000222559.jpg', 'width': 640, 'height': 426, 'channels': 3, 'mean_squared_error': 0.05674149061032864, 'peak_signal_noise_ratio': 60.59179620255149, 'structural_similarity': 0.9996361498119165, 'quality': 100}
(640, 474, 3) (640, 474)
after: (640, 474, 3) (640, 474, 3)
{'image': '000000222735.jpg', 'width': 474, 'height': 640, 'channels': 3, 'mean_squared_error': 0.05517207278481013, 'peak_signal_noise_ratio': 60.71

{'image': '000000341828.jpg', 'width': 612, 'height': 612, 'channels': 3, 'mean_squared_error': 0.0459118287838011, 'peak_signal_noise_ratio': 61.51155768714063, 'structural_similarity': 0.999735789336368, 'quality': 100}
(427, 640, 3) (427, 640)
after: (427, 640, 3) (427, 640, 3)
{'image': '000000355610.jpg', 'width': 640, 'height': 427, 'channels': 3, 'mean_squared_error': 0.047859338407494144, 'peak_signal_noise_ratio': 61.33113670099905, 'structural_similarity': 0.9995978007626215, 'quality': 100}
(427, 640, 3) (427, 640)
after: (427, 640, 3) (427, 640, 3)
{'image': '000000365095.jpg', 'width': 640, 'height': 427, 'channels': 3, 'mean_squared_error': 0.054625292740046835, 'peak_signal_noise_ratio': 60.75686583474397, 'structural_similarity': 0.9999193807792534, 'quality': 100}
(640, 603, 3) (640, 603)
after: (640, 603, 3) (640, 603, 3)
{'image': '000000365208.jpg', 'width': 603, 'height': 640, 'channels': 3, 'mean_squared_error': 0.05613339552238806, 'peak_signal_noise_ratio': 60.6

{'image': '000000474854.jpg', 'width': 424, 'height': 640, 'channels': 3, 'mean_squared_error': 0.055903596698113206, 'peak_signal_noise_ratio': 60.656406106578004, 'structural_similarity': 0.9996093661784441, 'quality': 100}
(491, 640, 3) (491, 640)
after: (491, 640, 3) (491, 640, 3)
{'image': '000000484978.jpg', 'width': 640, 'height': 491, 'channels': 3, 'mean_squared_error': 0.057360616089613035, 'peak_signal_noise_ratio': 60.544665535833, 'structural_similarity': 0.9996984460630002, 'quality': 100}
(566, 640, 3) (566, 640)
after: (566, 640, 3) (566, 640, 3)
{'image': '000000489842.jpg', 'width': 640, 'height': 566, 'channels': 3, 'mean_squared_error': 0.057925684628975266, 'peak_signal_noise_ratio': 60.502091854092804, 'structural_similarity': 0.9998635196325097, 'quality': 100}
(320, 500, 3) (320, 500)
after: (320, 500, 3) (320, 500, 3)
{'image': '000000504389.jpg', 'width': 500, 'height': 320, 'channels': 3, 'mean_squared_error': 0.05155, 'peak_signal_noise_ratio': 61.0085169124

In [48]:
def evaluate1(loss):
    orig = loss.replace('loss', 'orig')
    return evaluate(loss, orig)
    

In [55]:
len(results)

12400

In [56]:
df2 = pd.DataFrame(results)

In [59]:
fix_df(df2)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12400 entries, 0 to 12399
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   image                    12400 non-null  object 
 1   width                    12400 non-null  Int16  
 2   height                   12400 non-null  Int16  
 3   channels                 12400 non-null  Int16  
 4   mean_squared_error       12400 non-null  float64
 5   peak_signal_noise_ratio  12400 non-null  float64
 6   structural_similarity    12400 non-null  float64
 7   quality                  12400 non-null  int64  
dtypes: Int16(3), float64(3), int64(1), object(1)
memory usage: 593.5+ KB


In [61]:
df.set_index(['image', 'quality']).sort_index()

width  height  channels  mean_squared_error  \
image            quality                                                
000000000139.jpg 1          640     426         3          454.462972   
                 2          640     426         3          454.319354   
                 3          640     426         3          436.149269   
                 4          640     426         3          332.574298   
                 5          640     426         3          252.184591   
...                         ...     ...       ...                 ...   
000000581781.jpg 96         640     478         3            0.234861   
                 97         640     478         3            1.543861   
                 98         640     478         3            0.468701   
                 99         640     478         3            0.360817   
                 100        640     478         3            0.219530   

                          peak_signal_noise_ratio  structural_similarity  \
image            quality                                                   
000000000139.jpg 1                      21.555819               0.599887   
                 2                      21.557191               0.600065   
                 3                      21.734452               0.610393   
                 4                      22.911917               0.622467   
                 5                      24.113618               0.653344   
...                                           ...                    ...   
000000581781.jpg 96                     54.422695               0.998240   
                 97                     46.244721               0.994378   
                 98                     51.421845               0.997368   
                 99                     52.557939               0.997895   
                 100                    54.715862               0.998745   

                         orig_shape loss_shape  
image            quality                        
000000000139.jpg 1              NaN        NaN  
                 2              NaN        NaN  
                 3              NaN        NaN  
                 4              NaN        NaN  
                 5              NaN        NaN  
...                             ...        ...  
000000581781.jpg 96             NaN        NaN  
                 97             NaN        NaN  
                 98             NaN        NaN  
                 99             NaN        NaN  
                 100            NaN        NaN  

[500000 rows x 8 columns]

In [62]:
idf1 = df.set_index(['image', 'quality'])
idf2 = df2.set_index(['image', 'quality'])

In [63]:
idf1.loc[idf1.mean_squared_error.isna(), 'mean_squared_error'] = idf2.mean_squared_error

In [66]:
idf1.loc[bad_img[0]]

,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity,orig_shape,loss_shape
quality,,,,,,,,
1,<NA>,<NA>,<NA>,211.877179,NaN,NaN,"[512, 640, 3]","[512, 640]"
2,<NA>,<NA>,<NA>,211.821921,NaN,NaN,"[512, 640, 3]","[512, 640]"
3,<NA>,<NA>,<NA>,190.226074,NaN,NaN,"[512, 640, 3]","[512, 640]"
4,<NA>,<NA>,<NA>,136.732645,NaN,NaN,"[512, 640, 3]","[512, 640]"
5,<NA>,<NA>,<NA>,104.105035,NaN,NaN,"[512, 640, 3]","[512, 640]"
...,...,...,...,...,...,...,...,...
96,<NA>,<NA>,<NA>,0.051590,NaN,NaN,"[512, 640, 3]","[512, 640]"
97,<NA>,<NA>,<NA>,0.278641,NaN,NaN,"[512, 640, 3]","[512, 640]"
98,<NA>,<NA>,<NA>,0.117633,NaN,NaN,"[512, 640, 3]","[512, 640]"


In [67]:
idf1.loc[bad_img] = idf2

In [68]:
idf1.loc[bad_img[0]]

,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity,orig_shape,loss_shape
quality,,,,,,,,
1,640,512,3,211.877179,24.869962,0.692148,NaN,NaN
2,640,512,3,211.821921,24.871095,0.692155,NaN,NaN
3,640,512,3,190.226074,25.338103,0.711559,NaN,NaN
4,640,512,3,136.732645,26.772081,0.755938,NaN,NaN
5,640,512,3,104.105035,27.956086,0.783945,NaN,NaN
...,...,...,...,...,...,...,...,...
96,640,512,3,0.051590,61.005151,0.999501,NaN,NaN
97,640,512,3,0.278641,53.680357,0.998706,NaN,NaN
98,640,512,3,0.117633,57.425510,0.999184,NaN,NaN


In [73]:
idf1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 500000 entries, ('000000000139.jpg', 1) to ('000000581781.jpg', 100)
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   width                    500000 non-null  Int16  
 1   height                   500000 non-null  Int16  
 2   channels                 500000 non-null  Int16  
 3   mean_squared_error       500000 non-null  float64
 4   peak_signal_noise_ratio  500000 non-null  float64
 5   structural_similarity    500000 non-null  float64
 6   orig_shape               0 non-null       object 
 7   loss_shape               0 non-null       object 
dtypes: Int16(3), float64(3), object(2)
memory usage: 45.0+ MB


In [74]:
%%time
idf1.drop(['orig_shape', 'loss_shape'], axis='columns').to_csv(
    '../data/coco_degradation/val2017.csv',
    float_format='%.3f'
)

CPU times: user 2.78 s, sys: 120 ms, total: 2.91 s
Wall time: 2.85 s
